In [41]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

In [31]:
import numpy as np
import tensorflow as tf

def load_and_resize_images_in_batches(base_dir, subfolders, batch_size=100, target_size=(224, 224), output_file=None):
    # Initialize lists or arrays to hold batches
    resized_images = []
    labels = []

    for subfolder in subfolders:
        folder_path = os.path.join(base_dir, subfolder)
        print(f"Loading images from: {folder_path}")
        label = subfolder

        image_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        for i in range(0, len(image_files), batch_size):
            batch_files = image_files[i:i+batch_size]
            batch_images = []
            batch_labels = []
            
            for img_path in batch_files:
                try:
                    # Read and decode the image
                    img = tf.io.read_file(img_path)
                    img = tf.image.decode_image(img, channels=3)
                    
                    # Resize the image
                    img_resized = tf.image.resize(img, target_size)
                    
                    # Convert to NumPy array and add to batch
                    batch_images.append(np.array(img_resized))
                    batch_labels.append(label)
                    
                except Exception as e:
                    print(f"Error loading image {img_path}: {e}")
            
            # Convert batches to arrays and append to main list
            resized_images.extend(batch_images)
            labels.extend(batch_labels)

            # Optionally, save the batch to disk if output_file is provided
            if output_file:
                np.savez_compressed(output_file, X=np.array(resized_images), y=np.array(labels))
    
    return np.array(resized_images), np.array(labels)

# Define the base directory and subfolders
base_dir = r'DIAT-uSAT_dataset'
subfolders = [
    r"3_long_blade_rotor", 
    r"3_short_blade_rotor", 
    r"Bird", 
    r"Bird+mini-helicopter", 
    r"drone", 
    r"rc_plane", 
]

# Load and resize images in batches
X, y = load_and_resize_images_in_batches(base_dir, subfolders, batch_size=100)

print(f"Loaded {len(X)} images with shape {X.shape}.")
print(f"Loaded {len(y)} labels.")

Loading images from: DIAT-uSAT_dataset\3_long_blade_rotor
Loading images from: DIAT-uSAT_dataset\3_short_blade_rotor
Loading images from: DIAT-uSAT_dataset\Bird
Loading images from: DIAT-uSAT_dataset\Bird+mini-helicopter
Loading images from: DIAT-uSAT_dataset\drone
Loading images from: DIAT-uSAT_dataset\rc_plane
Loaded 4049 images with shape (4049, 224, 224, 3).
Loaded 4049 labels.


In [32]:
import numpy as np
import tensorflow as tf

def resize_images_in_batches(images, batch_size=100, target_size=(224, 224)):
    resized_images = []
    
    for i in range(0, len(images), batch_size):
        batch = images[i:i+batch_size]
        
        # Resize each image in the batch
        batch_resized = [tf.image.resize(img, target_size).numpy().astype(np.float16) for img in batch]
        
        # Append resized batch to the final list
        resized_images.extend(batch_resized)
    
    return np.array(resized_images)

# Process in batches
X_resized = resize_images_in_batches(X, batch_size=100)


In [33]:
# normalization with a rescaling factor of 1/255
X_resized = X_resized.astype('float32') / 255.0

In [38]:
# encoding labels into a numerical value
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(y)
# convert encoded labels into categorical format, where each class is represented by a binary vector
num_classes = len(label_encoder.classes_)
num_classes
categorical_labels = to_categorical(encoded_labels, num_classes)

In [35]:
# split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_resized, y, test_size=0.2, random_state=42)

In [36]:
# splitting of training and validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

#size of dataset
print(f"Training set size: {X_train.shape}, Training labels size: {y_train.shape}")
print(f"Validation set size: {X_val.shape}, Validation labels size: {y_val.shape}")
print(f"Testing set size: {X_test.shape}, Testing labels size: {y_test.shape}")

Training set size: (2429, 224, 224, 3), Training labels size: (2429,)
Validation set size: (810, 224, 224, 3), Validation labels size: (810,)
Testing set size: (810, 224, 224, 3), Testing labels size: (810,)


In [39]:
# data augmentation 
datagen = ImageDataGenerator(
    zoom_range = 0.2, # adds pixels
    brightness_range = [0.8, 1.2], # increasing intensity
    horizontal_flip = True, # random horizontal flipping
    fill_mode = 'nearest' # fill newly created pixels
)

In [40]:
# fit the datagen on training data
datagen.fit(X_train)

In [46]:
# defining model
model = models.Sequential()

# image input layer
model.add(layers.InputLayer(shape=(224, 224, 3)))

# convolutional layer (8 filters, 3x3, 0 padding)
model.add(layers.Conv2D(8, (3,3), padding='valid'))

# batch normalization layer 
model.add(layers.BatchNormalization())

# relu layer
model.add(layers.ReLU())

# max pooling layer (2x2, stride of 2)
model.add(layers.MaxPooling2D((2,2), strides=2))

# convolutional layer (16 filters, 3x3, 0 padding)
model.add(layers.Conv2D(16, (3, 3), padding='valid'))

# batch normalization layer 
model.add(layers.BatchNormalization())

# ReLU Layer
model.add(layers.ReLU())

# max pooling layer (2x2, stride of 2)
model.add(layers.MaxPooling2D((2,2), strides=2))

# convolutional layer (64 filters, 3x3, 0 padding)
model.add(layers.Conv2D(64, (3, 3), padding='valid'))

# batch normalization layer 
model.add(layers.BatchNormalization())

# ReLU layer
model.add(layers.ReLU())

# flatten the output before the fully connected layer
model.add(layers.Flatten())

# fully connected layer (4 neurons)
model.add(layers.Dense(4))

# softmax layer
model.add(layers.Softmax())

# compile model
model.compile(optimizer=Adam(learning_rate=0.00001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])